## Библиотеки

In [57]:
import numpy as np
import sys
from keras.datasets import mnist

## Данные

In [58]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [59]:
# Возьмем 1000 наблюдений и вытянем картинки в вектор и отцентрируем и нормируем. 
images, labels = x_train[0:1000].reshape(1000, 28*28) / 255, y_train[0:1000]
# Закодируем категориальный признак.
one_hot_labels = np.zeros((len(labels), 10))
for i, l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels
test_images = x_test.reshape(len(x_test), 28*28) / 255
test_labels = np.zeros((len(y_test), 10))
for i, l in enumerate(y_test):
    test_labels[i][l] = 1

In [60]:
# Зададим параметры.
alpha = 2
iterations = 300
hidden_size = 100
pixels_per_image = 784
num_labels = 10
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

In [61]:
# Функции активации.
def tanh(x):
    return np.tanh(x)

def tang2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

In [62]:
hidden_size = ((input_rows - kernel_rows) *
               (input_cols - kernel_cols)) * num_kernels
kernels = 0.02 * np.random.random(((kernel_rows * kernel_cols, num_kernels))) - 0.01
weights_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

In [63]:
def get_image_section(layer, row_from, row_to, col_from, col_to):
    sub_section = layer[ : , row_from : row_to, col_from : col_to]
    return sub_section.reshape(-1, 1, row_to-row_from, col_to-col_from)

In [64]:
for iter in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        
        batch_start, batch_end = (i * batch_size), ((i+1) * batch_size)
        layer_0 = images[batch_start : batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        
        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                    sect = get_image_section(layer_0,
                                            row_start,
                                            row_start+kernel_rows,
                                            col_start,
                                            col_start+kernel_cols)
                    sects.append(sect)

        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        
        kernel_ooutput = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_ooutput.reshape(es[0], -1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(layer_1.dot(weights_1_2))
        
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k : k+1]) == np.argmax(labels[batch_start+k : batch_start+k+1]))
            
        layer_2_delta = (layer_2 - labels[batch_start : batch_end]) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tang2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_ooutput.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
        
    test_correct_cnt = 0
    for j in range(len(test_labels)):
        layer_0 = test_images[j : j+1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        
        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                    sect = get_image_section(layer_0,
                                            row_start,
                                            row_start+kernel_rows,
                                            col_start,
                                            col_start+kernel_cols)
                    sects.append(sect)
    
        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        
        kernel_ooutput = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_ooutput.reshape(es[0], -1))
        layer_2 = layer_1.dot(weights_1_2)
        
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[j : j+1]))

    if (iter % 1 == 0):  
        sys.stdout.write('\n' + 'I : ' + str(iter) + ' Train-Acc : ' + str(correct_cnt/float(len(images/batch_size))) + ' Test-Acc : ' + str(test_correct_cnt/float(len(test_images/batch_size))))   


I : 0 Train-Acc : 0.15 Test-Acc : 0.2653
I : 1 Train-Acc : 0.2 Test-Acc : 0.2789
I : 2 Train-Acc : 0.205 Test-Acc : 0.3026
I : 3 Train-Acc : 0.213 Test-Acc : 0.3462
I : 4 Train-Acc : 0.216 Test-Acc : 0.3706
I : 5 Train-Acc : 0.264 Test-Acc : 0.4034
I : 6 Train-Acc : 0.282 Test-Acc : 0.4402
I : 7 Train-Acc : 0.314 Test-Acc : 0.4866
I : 8 Train-Acc : 0.339 Test-Acc : 0.5353
I : 9 Train-Acc : 0.373 Test-Acc : 0.5845
I : 10 Train-Acc : 0.407 Test-Acc : 0.6233
I : 11 Train-Acc : 0.422 Test-Acc : 0.6539
I : 12 Train-Acc : 0.455 Test-Acc : 0.681
I : 13 Train-Acc : 0.49 Test-Acc : 0.7001
I : 14 Train-Acc : 0.527 Test-Acc : 0.716
I : 15 Train-Acc : 0.536 Test-Acc : 0.726
I : 16 Train-Acc : 0.549 Test-Acc : 0.7377
I : 17 Train-Acc : 0.567 Test-Acc : 0.7437
I : 18 Train-Acc : 0.579 Test-Acc : 0.7543
I : 19 Train-Acc : 0.58 Test-Acc : 0.7617
I : 20 Train-Acc : 0.615 Test-Acc : 0.7681
I : 21 Train-Acc : 0.624 Test-Acc : 0.7733
I : 22 Train-Acc : 0.644 Test-Acc : 0.778
I : 23 Train-Acc : 0.642 Test